# Faltningssatsen i Fourieranalys

## Introduktion

Faltningssatsen är en fundamental sats i Fourieranalys som beskriver hur faltning i tidsdomänen övergår till multiplikation i frekvensdomänen. Denna sats har bred tillämpning inom signalbehandling, bildbehandling och fysik.

## Matematisk Bakgrund

### Definition av Faltning

För två funktioner f(t) och g(t) definieras faltningen som
\[
(f * g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t - \tau) d\tau
\]

### Faltningssatsen

Fouriertransformen av en funktion f(t) definieras som:
\[
\mathcal{F}\{f(t)\} = F(\omega) = \int_{-\infty}^{\infty} f(t)e^{-i\omega t} dt
\]


**Sats:** Om f(t) och g(t) har fouriertransformer F(ω) respektive G(ω), då gäller
\[
\mathcal{F}\{f(t) * g(t)\} = F(\omega) \cdot G(\omega)
\]

## Övning 

1. Nedan ges några par av funktioner. Plotta de enskilda funktionerna samt deras faltning. Använd `signal.convolve` för beräkning av faltningen.
2. Använd `np.fft.fft` för att beräkna fouriertransformen av faltningen och jämför med produkten av fouriertransformerna. Använd `np.fft.fftfreq` för att få en axel för frekvensdomänen. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Konfigurera plotting
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

# Använd subplots

plt.tight_layout()
plt.show()

## Exempel

In [ ]:
# Exempel 1: Rektangulära pulser 
t = np.linspace(-5, 5, 1000)
f1 = np.where((t >= -1) & (t <= 1), 1, 0)  # Rektangel från -1 till 1
g1 = np.where((t >= -0.5) & (t <= 0.5), 1, 0)  # Rektangel från -0.5 till 0.5

In [ ]:
# Exempel 2: Gaussiska pulser
f2 = np.exp(-t**2 / (2*0.5**2))  # Gaussisk puls med σ=0.5
g2 = np.exp(-t**2 / (2*0.3**2))  # Gaussisk puls med σ=0.3

In [ ]:
# Exempel 3: Sinus med olika frekvenser
f3 = np.sin(2*np.pi*2*t) * np.exp(-t**2/4)  # Sinus 2Hz med gaussisk envelop
g3 = np.sin(2*np.pi*5*t) * np.exp(-t**2/4)  # Sinus 5Hz med gaussisk envelop